In [1]:
#-------------------------Imports---------------------------
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.random import random

from pycbc.waveform import get_td_waveform
from pycbc import types, fft, waveform

from scipy.stats import uniform
from scipy import interpolate
from scipy.integrate import quad
from scipy import constants

import os
#from astropy.cosmology import Planck15
from astropy.cosmology import Planck18
from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits
from astropy.table import Table

from multiprocessing import Pool
import time

###########################
# CONSTANTS
###########################
H0GLOB= 69.32#67.9 #69
Om0GLOB=0.298
Xi0Glob =1.
clight = 2.99792458* 10**5#km/s
cosmoglob = Planck18
cosmofast = FlatLambdaCDM(H0=H0GLOB, Om0=Om0GLOB)
H0=cosmoglob.H(0).value
h=H0/100
#geometrization of masses
Msun=(1.98892)*(10**30)
solarmass_to_m=(constants.G*Msun)/((constants.c)**2)#G/c^2
Mpc_to_m=3.08567758128*(10**22) #this will be used later

#from lum distance to z
zz=np.linspace(0,20,10000)
distinterpol=cosmoglob.comoving_distance(zz).value*(1+zz)
z_of_dl=interpolate.interp1d(distinterpol,zz)

In [2]:
def phi_from_cart(x,y):
    if(x>0):
        phi=np.arctan(y/x)
    if ((x<0) and (y>=0)):
        phi=np.arctan(y/x)+np.pi
    if ((x<0) and (y<0)):
        phi=np.arctan(y/x)-np.pi
    return phi
def theta_from_cart(x,y,z):
    den=np.sqrt(x**2+y**2+z**2)
    theta=np.arccos(z/den)
    return theta

In [12]:
#-----------------Einstein Telescope Stuff------------------------------
#we will make two antenna functions, one is theresponse of a single interferometer
#the other will be the same as before, after a mean on psi

def ET_antenna_one_plus(cos_theta,phi,psi):
    prefactor=np.sqrt(3)/4
    A=(1+cos_theta**2)*(np.sin(2*phi))
    B=2*(cos_theta)*(np.cos(2*phi))
    ret=-prefactor*(A*np.cos(2*psi)+B*np.sin(2*psi))
    return ret
def ET_antenna_one_cross(cos_theta,phi,psi):
    prefactor=np.sqrt(3)/4
    A=(1+cos_theta**2)*(np.sin(2*phi))
    B=2*(cos_theta)*(np.cos(2*phi))
    ret=prefactor*(A*np.sin(2*psi)+B*np.cos(2*psi))
    return ret
def ET_antenna_two_plus(cos_theta,phi,psi):
    phi=phi+(2*np.pi/3)
    prefactor=np.sqrt(3)/4
    A=(1+cos_theta**2)*(np.sin(2*phi))
    B=2*(cos_theta)*(np.cos(2*phi))
    ret=-prefactor*(A*np.cos(2*psi)+B*np.sin(2*psi))
    return ret
def ET_antenna_two_cross(cos_theta,phi,psi):
    phi=phi+(2*np.pi/3)
    prefactor=np.sqrt(3)/4
    A=(1+cos_theta**2)*(np.sin(2*phi))
    B=2*(cos_theta)*(np.cos(2*phi))
    ret=prefactor*(A*np.sin(2*psi)+B*np.cos(2*psi))
    return ret
def ET_antenna_three_plus(cos_theta,phi,psi):
    phi=phi-(2*np.pi/3)
    prefactor=np.sqrt(3)/4
    A=(1+cos_theta**2)*(np.sin(2*phi))
    B=2*(cos_theta)*(np.cos(2*phi))
    ret=-prefactor*(A*np.cos(2*psi)+B*np.sin(2*psi))
    return ret
def ET_antenna_three_cross(cos_theta,phi,psi):
    phi=phi-(2*np.pi/3)
    prefactor=np.sqrt(3)/4
    A=(1+cos_theta**2)*(np.sin(2*phi))
    B=2*(cos_theta)*(np.cos(2*phi))
    ret=prefactor*(A*np.sin(2*psi)+B*np.cos(2*psi))
    return ret
#constant is a prefactor for the signal, mass must be the redshifted mass, angles are vectors
def ET_SNR_quad(f,constant,mass,dist,cos_i,ds_cos_theta,ds_phi,ds_psi):
    dist=dist*Mpc_to_m
    cos_theta=np.random.choice(ds_cos_theta)
    phi=np.random.choice(ds_phi)
    psi=np.random.choice(ds_psi)
    hplus_squared=((constant/dist)*(mass**(5/6))*(((1+cos_i**2)/2)**2)*f**(-7/6))**2
    hcross_squared=((constant/dist)*(mass**(5/6))*(cos_i**2)*f**(-7/6))**2
    numerator01=hplus_squared*(ET_antenna_one_plus(cos_theta,phi,psi))**2 + hcross_squared*(ET_antenna_one_cross(cos_theta,phi,psi))**2
    denom=ET_sn(f)**2
    first=4*numerator01/denom
    numerator02=hplus_squared*(ET_antenna_two_plus(cos_theta,phi,psi))**2 + hcross_squared*(ET_antenna_two_cross(cos_theta,phi,psi))**2
    second=4*numerator02/denom
    numerator03=hplus_squared*(ET_antenna_three_plus(cos_theta,phi,psi))**2 + hcross_squared*(ET_antenna_three_cross(cos_theta,phi,psi))**2
    third=4*numerator03/denom
    ret=np.array([first,second,third])
    return ret
def signal_noise_toint(f,signal):
    #numerator=((constant/dist)*(mass**(5/6))*(((1+cos**2)/2)**2)*f**(-7/6))**2
    numerator=signal(f)**2
    denom=(ET_sn(f)**2)
    return (numerator/denom)


In [24]:
#----function that perform the SNR-----------------
#---use this function in a loop, you must provide the angles for each ds
#read it from the EVA (catalogues) series.
def ET_SNR_Evaluation(ds_theta,ds_phi,ds_psi,hp,hc):
    temp_plus=quad(signal_noise_toint,ET_min_freq,ET_max_freq,args=(hp))[0]
    temp_cross=quad(signal_noise_toint,ET_min_freq,ET_max_freq,args=(hc))[0]
    first_prefactor=(ET_antenna_one_plus(ds_theta,ds_phi,ds_psi))**2+(ET_antenna_one_cross(ds_theta,ds_phi,ds_psi))**2
    firstSNR=(temp_plus+temp_cross)*first_prefactor*4
    second_prefactor=(ET_antenna_two_plus(ds_theta,ds_phi,ds_psi))**2+(ET_antenna_two_cross(ds_theta,ds_phi,ds_psi))**2
    secondSNR=(temp_plus+temp_cross)*second_prefactor*4
    third_prefactor=(ET_antenna_three_plus(ds_theta,ds_phi,ds_psi))**2+(ET_antenna_three_cross(ds_theta,ds_phi,ds_psi))**2
    thirdSNR=(temp_plus+temp_cross)*third_prefactor*4
    totSNR=np.sqrt(firstSNR+secondSNR+thirdSNR)
    return totSNR

In [5]:
#---------------Einstein Telescope Noise-----------------
ET_freq=np.loadtxt('ET-0000A-18_ETDSensitivityCurveTxtFile.txt',usecols=0)
ET_noise=np.loadtxt('ET-0000A-18_ETDSensitivityCurveTxtFile.txt',usecols=3)#This is the amplitude so must be squared
ET_sn=interpolate.interp1d(ET_freq,ET_noise)
ET_min_freq=ET_freq.min()
ET_max_freq=ET_freq.max()

In [23]:
#------------read the catalogue-------------------------------------
dat = Table.read('EVA01.fits', format='fits')
EVA = dat.to_pandas()#all good, is an only text fits
#print(EVA.columns)
ds_x=EVA['DS_x']
ds_y=EVA['DS_y']
ds_z=EVA['DS_z']
m1=EVA['M1']
m2=EVA['M2']
cos_inc=EVA['cos_orbital_incl']
inclin=np.arccos(cos_inc)
psi=EVA['psi']

In [28]:
#----------Aproximant-------------------------------
#EVA['Aproximant']='IMRPhenomD'
#t = Table.from_pandas(EVA)
#t.write('EVA02.fits', overwrite=True)

In [7]:
theta=np.zeros(EVA.shape[0])
phi=np.zeros(EVA.shape[0])
lum_distance=np.zeros(EVA.shape[0])
for i in range(EVA.shape[0]):
    phi[i]=phi_from_cart(ds_x[i],ds_y[i])
    theta[i]=theta_from_cart(ds_x[i],ds_y[i],ds_z[i])
    lum_distance[i]=np.sqrt(ds_x[i]**2+ds_y[i]**2+ds_z[i]**2)

In [21]:
SNR=np.zeros(EVA.shape[0])
start_time = time.time()
for i in range(3):
    FDhp,FDhc = waveform.get_fd_waveform(approximant="IMRPhenomD",
                             mass1=m1[i], mass2=m2[i],distance=lum_distance[i],inclination=inclin[i],
                                          delta_f=1.0/5000,f_lower=ET_min_freq,f_final=ET_max_freq)
    hp_interpol=interpolate.interp1d(FDhp.sample_frequencies,abs(FDhp),fill_value="extrapolate")
    hc_interpol=interpolate.interp1d(FDhc.sample_frequencies,abs(FDhc),fill_value="extrapolate")
    SNR[i]=ET_SNR_Evaluation(theta[i],phi[i],psi[i],hp_interpol,hc_interpol)
end_time= time.time()
print("--- %s seconds ---" % ((time.time() - start_time)))

<ipython-input-14-e84717f5100e>:6: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  temp_plus=quad(signal_noise_toint,ET_min_freq,ET_max_freq,args=(hp))[0]
<ipython-input-14-e84717f5100e>:7: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subrang

--- 263.00554037094116 seconds ---


In [22]:
print(SNR[0:9])

[ 47.56617886  94.99660899 168.10335855  70.82525956  89.00667008
 132.04828987 122.77244384  55.71635268  27.85337664]
